In [ ]:
'''
模糊综合评价的优点：
能较好地处理模糊性和不确定性问题，使评价结果更接近实际情况。
结果清晰，系统性强，能够提供全面的评价。
适用于复杂系统的综合评价和决策分析。
缺点；
权重的确定受到主观因素的影响
'''

In [4]:
#AHP模块
import numpy as np
from sklearn import preprocessing

'''
层级分析法
原理参考地址：https://www.cnblogs.com/yhll/p/9967726.html
方法结构：
      目标层          A
      准则层          B
      方案层          C
      
第一步：构成准则层矩阵   准则层矩阵就是指准则层B中N个因素对于目标A的的相对重要性
       重要性排序由1至9 准则层矩阵是对角矩阵，即Bi对Bj的重要性与Bj对Bi的重要性互为倒数
第二步：矩阵分解  将准则层矩阵直接求解特征值Val与特征向量Vec
        排序得到最大的特征值ValMax,并将对应的特征向量归一化，归一化后的特征向量是对应的权值
第三步：按照 （VALMAX-ROW）/（ROW-1）对照表格进行一致性检验，若通过则继续计算，若不通过需要重新建立矩阵

        后续方案层求解权重的方式重复步骤1-3，需要重复准则层因素个数N次
        
第四步：矩阵乘法  np.dot(Weight_B,Weight_C)
                即用准则层权值乘方案层权值矩阵，得到的结果是最终各个方案的成绩
                成绩对应最高则优
'''

class AHP():
    
    def __init__(self,Layer=3,B=3,C=8):
        '''
        Score :比较矩阵
        Layer :层数
        B     ：准则层个数
        C     ：方案层个数
        
        '''
        self.Layer = Layer
        self.B = B
        self.C = C
        
    def decompose(self,Data):
        
        # scaler = preprocessing.StandardScaler().fit(Data)#标准化
        # DataScale = scaler.transform(Data
        DataScale = Data#网上的过程似乎只需要对特征向量标准化
        
        # 特征值，特征向量矩阵（特征向量是列向量）
        val,vec = np.linalg.eig(DataScale)
        # 注意：这里复数只取了实数部分
        val = val.real.astype(float)
        vec = vec.real.astype(float)
        order = val.argsort()
        val = np.flip(val[order],0)
        vec = np.flip(vec[:,order],1)    #按列排列
        
        return val,vec
        
    def check(self,Data,Val):
        Row = Data.shape[0]
        RI = [0,0,0.58,0.90,1.12,1.24,1.32,1.41,1.45,1.49,1.51]
        CI = np.divide((Val-Row),(Row-1))
        CR = np.divide(CI,RI[Row-1])
        
        if CR > 0.1:
            raise Exception("Failed the consistency test.Please modify the matrix.")
        else:
            print('Consistency check passed')
            
        return CR
            
    def fit(self,Data_B,Data_C):
        
        B = self.B
        C = self.C
        
        CR_C = []
        Weight_C=[]
        Val_C = []
        #首先计算准则层
        Val_B,Vec_B = AHP.decompose(self,Data_B)
        Val_BMAX = Val_B[0]
        Weight_B = Vec_B[:,0]
        Weight_B = Weight_B.T
        Weight_B = np.divide(Weight_B,sum(Weight_B))
        CR_B = AHP.check(self,Data_B, Val_BMAX)
        
        self.CR_B = CR_B
        self.Weight_B = Weight_B
        self.Val_BMAX = Val_BMAX
        #计算方案层
        for i in range(B):
            if i == 0:
                Matrix_Current = Data_C[0:C,:]
                EndPoint = C
            else:
                StartPoint = EndPoint
                EndPoint = StartPoint+C
                Matrix_Current = Data_C[StartPoint:EndPoint,:]
                
            Val,Vec_C = AHP.decompose(self,Matrix_Current)
            Val_CMAX = Val[0]
            Val_C.append(Val_CMAX)
            Weight = Vec_C[:,0]
            Weight = Weight.T
            Weight = np.divide(Weight,sum(Weight))
            if i ==0:
                Weight_C = Weight
            else:
                Weight_C = np.vstack((Weight_C,Weight))
            CR = AHP.check(self,Matrix_Current, Val_CMAX)
            CR_C.append(CR)
                
        self.CR_C = CR_C
        self.Weight_C = Weight_C
        self.Val_C = Val_C
                
        
        Score = np.dot(Weight_B,Weight_C)
        
        return Score,self.Val_BMAX,self.Weight_B,self.CR_B,self.Val_C,self.Weight_C,self.CR_C,
                
    
if __name__ == '__main__':
    
    '''
    需要输入的参数：
    B            ：准则层个数   需要与外部对应的准则层矩阵.txt配合  
                   注意这个矩阵是对角矩阵   如果是N个准则  则是一个NXN的矩阵
    C            ：方案层个数   需要与外部对应的方案层矩阵.txt配合
                   每一个对应一个准则  都有一个方案层矩阵 如果是M个方案  则是一个MXM矩阵
                   在方案层矩阵.txt文件里面竖向存放了N个方案层矩阵
    
    
    因为你之前的论文有8个二级因素  所以默认的是准则层有8个 方案层三个     
    在运行时如果报错
    Failed the consistency test.Please modify the matrix.    
    那么代表没有通过一致性检验  需要重新构造矩阵 



    输出数据如下：
    Score       ：最终输出的三个方案的成绩
    Val_BMAX    ：准则层矩阵的最大特征值
    Weight_B    ：准则层矩阵的权向量（即对应最大特征值的归一化特征向量）
    CRB         ：准则层的一致化检验值，如果在右边变量管理器看不到，就在右下方控制台输入CRB回车就行
    Val_CMAX    ：方案层矩阵的最大特征值
    Weight_C    ：方案层矩阵的权向量（即对应最大特征值的归一化特征向量）
    CRC         ：方案层的一致化检验值，如果在右边变量管理器看不到，就在右下方控制台输入CRC回车就行
    
    这个矩阵的输入因为是分数 最好是你确定好 我帮你输
    '''
    Data_B = np.loadtxt('准则层矩阵.txt')
    Data_C = np.loadtxt('方案层矩阵.txt')
    
    AHP_Method = AHP(B=8,C=3)
    Score,Val_BMAX,Weight_B,CRB,Val_CMAX,Weight_C,CRC = AHP_Method.fit(Data_B,Data_C)

FileNotFoundError: 准则层矩阵.txt not found.

In [6]:
#模糊综合评价

import numpy as np
import AHP

'''
模糊综合评价法的解释
（1）指标
该模型有方案集 U = [u1,u2...um] 即提出的可能物流中心位置，m个
评价指标集合   V = [v1,v2...v4] 即合理，较合理，不太合理，差
因素指标集     D = [d1,d2...d4] 即包含四个一级指标
                  其中 d1 = [d11,d12]   d2 =[d21,d22,d23,d24]....  即一级指标下的二级指标
(2) 权重集
一级指标有4个  A = [a1,a2...a4] 是一级指标对应的权重
                  二级指标的权重为 Ai = [ai1,ai2...]
(3) 模糊评价集
邀请了m个专家  对所有的二级指标按照评价指标集合V评价
设对于因素集合 dij(二级指标)  分别属于 v1,v2,v3,v4的评价个数为  m1 m2 m3 m4
则对于该dij因素都模糊评价集为：

R = [m1/m... m4/m] = [rij1,rij2,rij3,rij4]     对于每一个二级因素 会产生一个 个数为len(V)的向量
则对于一级指标di 会形成一个模糊评价矩阵：

     | ri11 ri12...ri14|
Ri = |  .     .     .  |
     | rij1 rij2...rij4|
     
对于一级指标di  一级评判向量Bi,Bi是一个对于V都隶属度组成的向量:
    Bi = np.dot(AiRi) = [bi1,bi2,bi3...bi4]

由此得到二级评价矩阵

    | b11 b12 b13 b14|
R = | .    .   .   . |
    | b41 b42 b43 b44|

二次评价向量 B ：
    B = np.dot(AR)
'''


class fuzzy():
    
    def __init__(self,Score,U=4,V=4,num_D=4,list_D=[2,4,2,2]):
        '''
        Score :打分矩阵
        U     ：考虑都选址个数
        V     ：评价指标个数
        num_D : 一级指标个数
        list_D: 一级指标分别的二级指标个数

        
        '''
        self.Score = Score
        self.U = U
        self.V = V
        self.num_D = num_D
        self.list_D = list_D

    def Get_weight(self):
        '''
        获取权重
        '''
        Score = self.Score
        V = self.V
        num_D = self.num_D
        list_D = self.list_D
        Fir_sum = 0
        Fireach_score = [0]*num_D
        weight = []
        
        for i in range(num_D):         #一级
            for j in range(V):
                if i < num_D:
                    Fir_sum += (V-j)*Score[i,j]
                    Fireach_score[i] += (V-j)*Score[i,j]     
        Fir_weight = np.divide(Fireach_score,Fir_sum)
        weight.append(Fir_weight)
        
        up_limit = num_D
        for model in range(len(list_D)):  #二级
            Sec_sum = 0
            Seceach_score = [0]*list_D[model]
            low_limit = up_limit
            up_limit = low_limit + list_D[model]
            for i in range(Score.shape[0]):
                for j in range(V):
                    if i >=low_limit and i<up_limit :
                        Sec_sum += (V-j)*Score[i,j]
                        Seceach_score[i-low_limit] += (V-j)*Score[i,j]
            Sec_weight = np.divide(Seceach_score,Sec_sum)
            weight.append(Sec_weight)
        
        self.Weight = weight
        return weight
    
    def GetWeight_AHP(self,Matrix_B,Matrix_C):
        
        self.Matrix_B= Matrix_B
        self.Matrix_C= Matrix_C
        V = self.V
        num_D = self.num_D
        list_D = self.list_D
        U = self.U
        weight = []
        CR_list = []
        AHP_Method = AHP.AHP()
        #计算一级指标权重
        Fir_Val,Fir_Vec = AHP_Method.decompose(Matrix_B)
        Fir_ValMAX = Fir_Val[0]
        Fir_weight = Fir_Vec[:,0]
        Fir_weight = Fir_weight.T
        Fir_weight = np.divide(Fir_weight,sum(Fir_weight))
        CR_First = AHP_Method.check(Matrix_B, Fir_ValMAX)
        CR_list.append(CR_First)
        weight.append(Fir_weight)
        #计算二级指标权重
        for model in range(len(list_D)):  #二级
            
            # if model == 0:
            #     Matrix_Current = Matrix_C[0:U,:]
            #     EndPoint = U
            # else:
            #     StartPoint = EndPoint
            #     EndPoint = StartPoint+U
            #     Matrix_Current = Matrix_C[StartPoint:EndPoint,:]
            Matrix_Current = Matrix_C[model]
                
            Sec_Val,Sec_Vec = AHP_Method.decompose(Matrix_Current)
            Sec_ValMAX = Sec_Val[0]
            Sec_weight = Sec_Vec[:,0]
            Sec_weight = Sec_weight.T
            Sec_weight = np.divide(Sec_weight,sum(Sec_weight))
            CR_Sec = AHP_Method.check(Matrix_Current, Sec_ValMAX)
            CR_list.append(CR_Sec)
            weight.append(Sec_weight)
            
        self.CR=CR_list
        self.Weight = weight
        return weight
        
    
    def Get_R(self):
        
        '''
        评价矩阵
        '''
        Score = self.Score
        V = self.V
        num_D = self.num_D
        list_D = self.list_D
        R_list = []
        
        for i in range(num_D):         #一级
            sum_i = 0
            for j in range(V):
                if i < num_D:
                    sum_i += Score[i,j]
            score_i = np.divide(Score[i,:],sum_i)
            if i ==0:
                R = score_i
            elif i < num_D:
                R = np.vstack((R,score_i))
        R_list.append(R)
        up_limit = num_D
        for model in range(len(list_D)):  #二级
            low_limit = up_limit
            up_limit = low_limit + list_D[model]
            for i in range(Score.shape[0]):
                sum_i = 0
                for j in range(V):
                    if i >= low_limit and i< up_limit :
                        sum_i += Score[i,j]
                score_i = np.divide(Score[i,:],sum_i)
                if i == low_limit:
                    R = score_i
                if i > low_limit and i< up_limit :
                    R = np.vstack((R,score_i))
            R_list.append(R)
        
        self.R = R_list
        return R_list        
    
    def Get_B(self):
        '''
        评价向量
        '''
        B_list = []
        weight = self.Weight
        R = self.R
        
        for i in range(len(weight)):
            B = np.dot(weight[i],R[i])
            B_list.append(B)
        
        R_Sec = B_list[1]
        for m in range(len(B_list)-2):
            R_Sec = np.vstack((R_Sec,B_list[m+2]))
        
        B_Sec = np.dot(weight[0].T,R_Sec)
        
        self.B_list = B_list
        self.R_Sec = R_Sec
        self.B_Sec = B_Sec
        return B_list,R_Sec,B_Sec
    
    
        
    
        
if __name__ == '__main__':
    '''
    输入：
    test 是投票数据矩阵    在代码文件夹里面有个test.txt文件  可以直接改投票数据
    U    是指明有几个待比较的地点  没啥用
    V    投标标准个数  默认4
    num_D一级指标个数  默认3
    list_D 二级指标个数  默认 2 3 3
    num_D 和list_D必须和test一致
    输出：
    weight  根据投票结果得到的每个指标的权重  编号0是一级其余为二级
    R_list  一级评价矩阵， 编号0是一级其余是二级
    B_list  一级评价向量   编号0是一级其余是二级
    R_Sec   二级评价矩阵
    B_Sec   二级评价向量   最后的B_Sec就是模糊评价结果  0 1 2 3 分别是很合理 一般合理  不合理  很差
    '''
#    Score = np.loadtxt('Score.txt')
    test = np.loadtxt('test4.txt')          #在这里改数据输入
    Address_=['二级指标矩阵1.txt',
              '二级指标矩阵2.txt',
              '二级指标矩阵3.txt',]
    
    Data_C = []
    Data_B = np.loadtxt('一级指标矩阵.txt')
    Data = np.loadtxt('二级指标矩阵1.txt')
    Data_C.append(Data)
    Data = np.loadtxt('二级指标矩阵2.txt')
    Data_C.append(Data)
    Data = np.loadtxt('二级指标矩阵3.txt')
    Data_C.append(Data)

    
    
    Fuzzy = fuzzy(test,U=4,V=4,num_D=3,list_D=[2,3,3])   #在这里改输入
    #weight = Fuzzy.Get_weight()
    weight = Fuzzy.GetWeight_AHP(Data_B,Data_C)
    R_list = Fuzzy.Get_R()
    B_list,R_Sec,B_Sec = Fuzzy.Get_B()

ModuleNotFoundError: No module named 'AHP'